<a href="https://colab.research.google.com/github/utsav-195/fingerprint-recognition-using-siamese-network/blob/master/fingerprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# importing the required libraries
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

from tensorflow import keras
from PIL import Image

In [2]:
def augment(folder,file):
  filename = folder + "/" + file
  # loading in the images
  image = imageio.imread(filename)

  flip_vr=iaa.Flipud(p=1.0)
  flip_vr_image= flip_vr.augment_image(image)

  save_filename = folder + "/" + file.split(".")[0] + "_flipped.png"
  cv2.imwrite(save_filename, flip_vr_image)

  rotate = iaa.Affine(rotate=(50, -50))
  rotated_image = rotate.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_rotated1.png"
  cv2.imwrite(save_filename, rotated_image)

  rotate = iaa.Affine(rotate=(50, -50))
  rotated_image = rotate.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_rotated2.png"
  cv2.imwrite(save_filename, rotated_image)

  crop = iaa.Crop(percent=(0, 0.3)) # crop image
  crop_image=crop.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_cropped1.png"
  cv2.imwrite(save_filename, crop_image)

  crop = iaa.Crop(percent=(0, 0.3)) # crop image
  crop_image=crop.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_cropped2.png"
  cv2.imwrite(save_filename, crop_image)

  contrast=iaa.GammaContrast(gamma=2.0)
  contrast_image =contrast.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_bright1.png"
  cv2.imwrite(save_filename, contrast_image)

  contrast=iaa.GammaContrast(gamma=1.4)
  contrast_image =contrast.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_bright2.png"
  cv2.imwrite(save_filename, contrast_image)

  blur = iaa.GaussianBlur(sigma=4.0)
  blur_image=blur.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_blur1.png"
  cv2.imwrite(save_filename, blur_image)

  blur = iaa.GaussianBlur(sigma=2.0)
  blur_image=blur.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_blur2.png"
  cv2.imwrite(save_filename, blur_image)

In [3]:
dimen = 224

dir_path = "/content/drive/My Drive/fingerprint/images/"
out_path = "/content/drive/My Drive/fingerprint/processed_images/"
model_path = "/content/drive/My Drive/fingerprint/model/"

In [ ]:
sub_dir_list = os.listdir(dir_path)

images = []
for i in range(len(sub_dir_list)):
  # image_names = os.listdir(os.path.join(dir_path, sub_dir_list[i]))
  # augment(os.path.join(dir_path, sub_dir_list[i]),image_names[0])
  image_names = os.listdir(os.path.join(dir_path, sub_dir_list[i]))
  for image_path in image_names:
    path = os.path.join(dir_path, sub_dir_list[i], image_path )
    try:
      print(path)
      image = Image.open(path)
      resize_image = image.resize((dimen, dimen))
      array_ = list()
      for x in range(dimen):
        sub_array = list()
        for y in range(dimen):
          sub_array.append(resize_image.load()[x, y])
        array_.append(sub_array)
      image_data = np.array(array_)
      image = np.array(np.reshape(image_data, (dimen, dimen, 3))) / 255
      images.append(image)
    except:
      print('WARNING : File {} could not be processed.'.format(path))

images = np.array(images)

In [ ]:
samples_1 = []
samples_2 = []
labels = []

for i in range(len(images)):
  for j in range(len(images)):
    samples_1.append(images[i])
    samples_2.append(images[j])
    t = i - i%10 +10
    if t - 10 <= j < t:
      labels.append(1)
    else:
      labels.append(0)

X1 = np.array(samples_1)
X2 = np.array(samples_2)
Y = np.array(labels)

np.save( '{}/x1.npy'.format( out_path ), X1 )
np.save( '{}/x2.npy'.format( out_path ), X2 )
np.save( '{}/y.npy'.format( out_path ) , Y )

In [7]:
from tensorflow.keras import models , optimizers , losses ,activations , callbacks
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from PIL import Image
import tensorflow as tf
import time
import os
import numpy as np

class Recognizer (object) :

	def __init__( self ):

		# tf.logging.set_verbosity( tf.logging.ERROR )

		self.__DIMEN = 224

		input_shape = ( (self.__DIMEN**2) * 3 , )
		convolution_shape = ( self.__DIMEN , self.__DIMEN , 3 )
		kernel_size_1 = ( 10 , 10 )
		kernel_size_2 = ( 7 , 7 )
		kernel_size_3 = ( 4 , 4 )
		# kernel_size_2 = ( 7 , 7 )
	
		# pool_size_1 = ( 3 , 3 )
		# pool_size_2 = ( 2 , 2 )
		strides = 1

		seq_conv_model = [

			Reshape( input_shape=input_shape , target_shape=convolution_shape),

			Conv2D( 64, kernel_size=kernel_size_1 , strides=strides , activation='relu' ),
			# Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation='relu'),
			MaxPooling2D(),

			Conv2D( 128, kernel_size=kernel_size_2 , strides=strides , activation='relu'),
			# Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation='relu'),
			MaxPooling2D(),

      Conv2D( 128, kernel_size=kernel_size_3 , strides=strides , activation='relu'),
			# Conv2D( 128, kernel_size=kernel_size_3 , strides=strides , activation='relu'),
			MaxPooling2D(),

			Conv2D( 256, kernel_size=kernel_size_3 , strides=strides , activation='relu'),
			# Conv2D( 128, kernel_size=kernel_size_3 , strides=strides , activation='relu'),

			Flatten(),

			Dense( 1024 , activation='relu' ),
	 		Dense( 1024 , activation=activations.sigmoid )

		]

		seq_model = tf.keras.Sequential( seq_conv_model )

		input_x1 = Input( shape=input_shape )
		input_x2 = Input( shape=input_shape )

		output_x1 = seq_model( input_x1 )
		output_x2 = seq_model( input_x2 )

		distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
		outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
		self.__model = models.Model( [ input_x1 , input_x2 ] , outputs )

		self.__model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.00003))



	def fit(self, X, Y ,  hyperparameters  ):
		initial_time = time.time()
		self.__model.fit( X  , Y ,
						 batch_size=hyperparameters[ 'batch_size' ] ,
						 epochs=hyperparameters[ 'epochs' ] ,
						 callbacks=hyperparameters[ 'callbacks'],
						 validation_data=hyperparameters[ 'val_data' ]
						 )
		final_time = time.time()
		eta = ( final_time - initial_time )
		time_unit = 'seconds'
		if eta >= 60 :
			eta = eta / 60
			time_unit = 'minutes'
		self.__model.summary( )
		print( 'Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit ) )


	def prepare_images_from_dir( self , dir_path , flatten=True ) :
		images = list()
		images_names = os.listdir( dir_path )
		for imageName in images_names :
			# print(imageName)
			image = Image.open(dir_path + imageName)
			resize_image = image.resize((self.__DIMEN, self.__DIMEN))
			array = list()
			for x in range(self.__DIMEN):
				sub_array = list()
				for y in range(self.__DIMEN):
					sub_array.append(resize_image.load()[x, y])
				array.append(sub_array)
			image_data = np.array(array)
			image = np.array(np.reshape(image_data,(self.__DIMEN, self.__DIMEN, 3)))
			images.append(image)

		if flatten :
			images = np.array(images)
			return images.reshape( ( images.shape[0]  , self.__DIMEN**2 * 3  ) ).astype( np.float32 )
		else:
			return np.array(images)



	def evaluate(self , test_X , test_Y  ) :
		return self.__model.evaluate(test_X, test_Y)


	def predict(self, X  ):
		predictions = self.__model.predict( X  )
		return predictions


	def summary(self):
		self.__model.summary()

	def save_model(self , file_path ):
		self.__model.save(file_path )


	def load_model(self , file_path ):
		self.__model = models.load_model(file_path)

In [5]:
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
import numpy as np
import time

data_dimension = 224

X1 = np.load('{}/x1.npy'.format( out_path ))
X2 = np.load('{}/x2.npy'.format( out_path ))
Y = np.load('{}/y.npy'.format( out_path ))

X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )

recognizer = Recognizer()

from keras.callbacks import ModelCheckpoint

# checkpointer to save the best model
checkpointer = ModelCheckpoint(filepath='{}/model_test5.h5'.format(model_path), monitor='loss', verbose=1, save_best_only=True)

parameters = {
    'batch_size' : 50,
    'epochs' : 30,
    'callbacks' : checkpointer,
    'val_data' : None
}

recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)

(2500, 150528)
(2500, 150528)
(2500,)
Epoch 1/20
50/50 [==============================] - ETA: 0s - loss: 0.5580
Epoch 00001: loss improved from inf to 0.55796, saving model to /content/drive/My Drive/fingerprint/model/model_test4.h5
50/50 [==============================] - 30s 603ms/step - loss: 0.5580
Epoch 2/20
50/50 [==============================] - ETA: 0s - loss: 0.4466
Epoch 00002: loss improved from 0.55796 to 0.44657, saving model to /content/drive/My Drive/fingerprint/model/model_test4.h5
50/50 [==============================] - 33s 655ms/step - loss: 0.4466
Epoch 3/20
50/50 [==============================] - ETA: 0s - loss: 0.3916
Epoch 00003: loss improved from 0.44657 to 0.39160, saving model to /content/drive/My Drive/fingerprint/model/model_test4.h5
50/50 [==============================] - 33s 662ms/step - loss: 0.3916
Epoch 4/20
50/50 [==============================] - ETA: 0s - loss: 0.3313
Epoch 00004: loss improved from 0.39160 to 0.33134, saving model to /content/d

In [9]:
recognizer = Recognizer()
recognizer.load_model('{}/model_test5.h5'.format(model_path))

test_images = recognizer.prepare_images_from_dir("/content/drive/My Drive/fingerprint/test_images/")
class_images = "/content/drive/My Drive/fingerprint/images/"

scores = []
labels = []
for image in test_images:
    # print(image)
    label = []
    score = []
    for class_name in os.listdir(class_images):
      samples = recognizer.prepare_images_from_dir("/content/drive/My Drive/fingerprint/images/"+class_name+"/")
      for sample in samples:
        # print(sample)
        image, sample = image.reshape((1, -1)), sample.reshape((1 ,-1))
        score.append(recognizer.predict([image, sample])[0])
        label.append(class_name)
    labels.append(label)
    scores.append(score)

scores = np.array(scores)
labels = np.array(labels)

for i in range(test_images.shape[0]) :
    index = np.argmax(scores[i])
    label_ = labels[i][index]
    print( 'IMAGE {} is {} with confidence of {}'.format( i+1  , label_ , scores[i][index][0] ) )

IMAGE 1 is thumb with confidence of 0.9975006580352783
IMAGE 2 is first with confidence of 0.9981439709663391
IMAGE 3 is middle with confidence of 0.9968104958534241
IMAGE 4 is pinky with confidence of 0.9980166554450989
IMAGE 5 is ring with confidence of 0.9958359003067017
IMAGE 6 is thumb with confidence of 0.9984050989151001
IMAGE 7 is pinky with confidence of 0.977330207824707
